In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras import backend as K
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, r2_score
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences

In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation,Bidirectional, GlobalMaxPool1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
import os, string
from tensorflow.keras import callbacks
from keras.callbacks import Callback
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))
from sklearn.preprocessing import MinMaxScaler

In [ ]:
EMBEDDING_FILE_FASTTEXT="D:/FastText/crawl-300d-2M.vec/crawl-300d-2M.vec"
EMBEDDING_FILE_TWITTER="D:/glove.twitter.27B/glove.twitter.27B.200d.txt"
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index_ft = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE_FASTTEXT,encoding='utf-8'))
embeddings_index_tw = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE_TWITTER,encoding='utf-8'))

In [ ]:

cols_features = ['count_word','capitals', 'count_unique_word',  'count_punctuations','word_unique_percent' ,'punct_percent'
                 , 'num_exclamation_marks', 'num_question_marks', 'you_count','mentions','smilies' ,'symbols']


def get_features(df):
    
    df['count_word']=df["tweet"].apply(lambda comment: len(str(comment).split()))
    df['capitals'] = df["tweet"].apply(lambda comment: sum(1 for c in str(comment) if c.isupper()))
    df['count_unique_word']=df["tweet"].apply(lambda comment: len(set(str(comment).split())))
    df["count_punctuations"] =df["tweet"].apply(lambda comment: len([c for c in str(comment) if c in  string.punctuation]))#try remove
    df['word_unique_percent']=df['count_unique_word']*100/df['count_word']
    df['punct_percent']=df['count_punctuations']*100/df['count_word']
    df['num_exclamation_marks'] = df['tweet'].apply(lambda comment: str(comment).count('!'))
    df['num_question_marks'] = df['tweet'].apply(lambda comment:  str(comment).count('?'))
    df['you_count'] = df['tweet'].apply(lambda comment: sum( str(comment).count(w) for w in ('you', 'You', 'YOU')))
    df['mentions'] = df['tweet'].apply(lambda comment: str(comment).count("@"))
    df['smilies'] = df['tweet'].apply(lambda comment: sum(str(comment).count(w) for w in (':-)', ':)', ';-)', ';)')))#try remove
    df['symbols'] = df['tweet'].apply(lambda comment: sum(str(comment).count(w) for w in '*&#$%“”¨«»®´·º½¾¿¡§£₤‘’'))
    
    scaler = MinMaxScaler().fit(df[cols_features])
    df[cols_features] = scaler.transform(df[cols_features])
    return df

In [ ]:
train_df =  pd.read_csv('C:/Users/Lenovo/Desktop/GP Code/CleaneddroppedCivil60K2/CleaneddroppedCivil60K2.csv')

In [ ]:
train_df = get_features(train_df)
train_df["toxicity"] = train_df["toxicity"].apply(lambda x: 1 if x>=0.5 else 0)
train_df.head(5)

In [ ]:
cols = ['lemmatize_text','count_word','capitals', 'count_unique_word',  'count_punctuations','word_unique_percent' ,'punct_percent'
                 , 'num_exclamation_marks', 'num_question_marks', 'you_count','mentions','smilies' ,'symbols']
x_train, x_test, y_train, y_test = train_test_split(train_df[cols], train_df['toxicity'].values, test_size=0.2, shuffle=True)

In [ ]:
X_test = x_test['lemmatize_text'].astype(str)
X = x_train['lemmatize_text'].astype(str)
X_feat = x_train[cols_features]
Y = y_train
Test_feat = x_test[cols_features]


In [ ]:

tokenizer = Tokenizer( oov_token = True)
tokenizer.fit_on_texts(list(X))
X_train_sequence = tokenizer.texts_to_sequences(X)
X_test_sequence = tokenizer.texts_to_sequences(X_test)
max_features = len(tokenizer.word_index) + 1


In [ ]:
def FindMaxLength(lst):
    maxList = max((x) for x in lst)
    maxLength = max(len(x) for x in lst )
  
    return maxList, maxLength

In [ ]:
# find the longest sequence
_, maxlen1 = FindMaxLength(X_train_sequence)
_, maxlen2 = FindMaxLength(X_test_sequence )
maxlen = max(maxlen2 ,maxlen1)


In [ ]:
from itertools import cycle

def list_padding(lst , maxlen):
    lstNew = lst.copy()
    myiter = cycle(lstNew)
    for _ in range(maxlen):
        lstNew.append(next(myiter))
    return lstNew

In [ ]:
def custom_padding(lst, maxlength):
    new =[]
    for  i in  range(len(lst)):
        length =  maxlength - len(lst[i])
        my_padding = list_padding(lst[i] , length)
        new.append( my_padding)
    return new    

In [ ]:

padTrain =custom_padding(X_train_sequence, maxlen)
padTest = custom_padding(X_test_sequence, maxlen)
x_train_pad = pad_sequences(padTrain )
x_test_pad  = pad_sequences(padTest )


In [ ]:
print(max_features)

In [ ]:
word_index = tokenizer.word_index
nb_words = max_features
embedding_matrix = np.zeros((nb_words,500))
dim =500

something_tw = embeddings_index_tw.get("something")
something_ft = embeddings_index_ft.get("something")

something = np.zeros((500,))
something[:300,] = something_ft
something[300:500,] = something_tw
# we define here embedding vector for word "something" whenever the model founds word doesnot have vector representing 
#in both fasttext and glove


def embed_word(embedding_matrix,i,word):
    embedding_vector_ft = embeddings_index_ft.get(word)
    if embedding_vector_ft is not None: 
        embedding_matrix[i,:300] = embedding_vector_ft
        embedding_vector_tw = embeddings_index_tw.get(word)
        if embedding_vector_tw is not None:
            embedding_matrix[i,300:500] = embedding_vector_tw
#here we define to arrange fasttext first 300 and then glove the second 200
            
for word, i in word_index.items():            
    if embeddings_index_ft.get(word) is not None:
        embed_word(embedding_matrix,i,word)
    else:
        embedding_matrix[i] = something
# we loop over all unique words in word-index_items  
# if the there is unique word doenot have glove nor fasttext vector we put "something" vector representing for that word

In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
inp = Input(shape = (maxlen,))
x = Embedding(max_features, dim , weights = [embedding_matrix], trainable = False)(inp)
x = Bidirectional(GRU(20, return_sequences=True))(x)
x = attention()(x)
features = Input(shape=(len(cols_features),))
conc = concatenate([x, features])
x= Flatten(name='Flatten')(x)
conc = Dropout(0.2)(x)
preds = Dense(1, activation="sigmoid")(conc)
model = Model(inputs=[inp, features], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer=Adam(clipvalue=1, clipnorm=1), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
K.clear_session()
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=239)
acc_score = []
X_Flod = x_train_pad
y_Flod = Y
feat_Flod = X_feat.values

for train_index , test_index in kf.split(X):

    X_train , X_valid = X_Flod[train_index],  X_Flod[test_index]
    y_train , y_valid =y_Flod[train_index] ,y_Flod[test_index]
    feat_Train ,feat_test =  feat_Flod[train_index], feat_Flod[test_index]
    K.clear_session()
    hist = model.fit([X_train,feat_Train],y_train,epochs=1,batch_size = 32,verbose=1)
    pred_values = model.predict([X_valid,feat_test],batch_size = 32,verbose=1)
    pred_values*=100
    pred_values=[1 if i >50  else 0 for i in pred_values ]
    acc = accuracy_score(pred_values ,y_valid)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

In [ ]:
prediction = model.evaluate( [x_test_pad,Test_feat],y_test,batch_size = 32,verbose=1)
print(f'Test loss: {prediction[0]} / Test accuracy: {prediction[1]}')

In [ ]:
test_df =  pd.read_csv('C:/Users/Lenovo/Desktop/GP Code/CleanTest.csv')
test_df = get_features(test_df )

In [ ]:
test_df.head(10)

In [ ]:
testX = test_df['lemmatize_text'].astype(str)
testY =test_df['label']
testFeat = test_df[cols_features]

In [ ]:
testX_sequence = tokenizer.texts_to_sequences(testX)


In [ ]:
padTesting=custom_padding(testX_sequence, maxlen)
testX_pad  = pad_sequences(padTesting)



In [ ]:
pred_v = model.predict([testX_pad,testFeat])
pred_v*=100

In [ ]:
pred_v=[1 if i >20  else 0 for i in pred_v ]
print("predictied values\n")
print(pred_v)

In [ ]:
print("expected values\n")
print("[0, 1, 0, 0, 1, 1, 1 , 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0]"   )          